In [102]:
import warnings
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes, fetch_openml,load_iris,fetch_california_housing
from sklearn.feature_selection import mutual_info_regression, f_regression, RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
RepeatedStratifiedKFold, 
cross_val_score, 
train_test_split, 
GridSearchCV,
cross_val_predict, 
learning_curve, 
validation_curve)
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error,zero_one_loss, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.datasets import make_circles, make_moons, make_blobs
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from os.path import join as pjoin
#sharper plots
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
from sklearn.linear_model import (LogisticRegression, LogisticRegressionCV,
                                  SGDClassifier)
from PIL import Image
import random
warnings.filterwarnings("ignore")
from shutil import copyfile
from sklearn.svm import SVC
import openpyxl

In [103]:
steamspy_path = r"C:\Users\Acer\Desktop\spbu_ml_2025\data\games-review-sentiment\output_steamspy.csv"
steamstore_path = r"C:\Users\Acer\Desktop\spbu_ml_2025\data\games-review-sentiment\steam_store_data_2024.csv"
test_path = r"C:\Users\Acer\Desktop\spbu_ml_2025\data\games-review-sentiment\test.csv"
train_path = r"C:\Users\Acer\Desktop\spbu_ml_2025\data\games-review-sentiment\train.csv"

df_test = pd.read_csv(test_path)
df_train = pd.read_csv(train_path)
df_steamspy = pd.read_csv(steamspy_path)
df_steamstore = pd.read_csv(steamstore_path)

df_test.sample(5)


,id,app_id,content,author_id
12192,12193,300,Hitler,76561198961943535
2650,2651,400,This game and many others used to have full co...,76561198384199183
15498,15499,420,I don't want HL3 I want HL0,76561198047566695
49387,49388,280,I disliked that the maps are really messed up ...,76561198162475156
60242,60243,620,"Portals good, wheatly good, glados good, cave ...",76561199589768443


In [104]:
df_train.sample(5)


,id,app_id,content,author_id,is_positive
58348,58349,20,i like team fortress 2 i spent hours of it but...,76561198131000933,1
130802,130803,730,"Hayatımda gördüğüm en rezil, en ucube şirket v...",76561198807087439,0
14264,14265,730,Biggest money grab in the history of money grabs,76561198438604737,0
97484,97485,240,amazing shooter game it can compete with call ...,76561199421289443,1
44179,44180,20,Fun.,76561197967506018,1


In [110]:
sample = df_steamstore.copy()

# Оставляем только те строки, где значение "price" встречается один раз в выборке
unique_prices = sample["price"].value_counts()
unique_sample = sample[sample["price"].isin(unique_prices[unique_prices == 1].index)]
unique_sample

,title,description,price,salePercentage,recentReviews,allReviews
5,Headbangers: Rhythm Royale,"Headbangers is a light-hearted, Rhythm Battle ...",$13.39,-33%,Very Positive,Very Positive
9,Boxes: Lost Fragments,"As a legendary thief, your next assignment lur...",$13.49,-10%,Very Positive,Very Positive
10,HELLCARD,Hellcard is a unique cooperative deck builder ...,$15.79,-21%,Very Positive,Very Positive
12,BlazBlue Entropy Effect,"Artistic and Magnificent Combat Action, incred...",$17.15,-22%,Mostly Positive,Mostly Positive
19,Hell Let Loose,Join the ever expanding experience of Hell Let...,$42.24,-35%,Very Positive,Very Positive
20,HELLDIVERS™ 2 - Upgrade to Super Citizen Edition,NaN,$59.99,NaN,NaN,NaN
42,Banishers: Ghosts of New Eden,Hunt ghosts as two memorable characters in a s...,$49.99,NaN,NaN,NaN
44,Tomb Raider I-III Remastered Starring Lara Croft,Play the original three Tomb Raider adventures...,$26.99,-10%,NaN,NaN
56,Days Gone,"Ride and fight into a deadly, post pandemic Am...",$12.49,-75%,Very Positive,Very Positive
63,Overcooked! 2,Overcooked returns with a brand-new helping of...,$6.24,-75%,Very Positive,Very Positive


In [ ]:
df_steamspy.sample(5)

,appid,name,owners
767,863550,HITMAN 2,"5,000,000 .. 10,000,000"
373,280790,Creativerse,"1,000,000 .. 2,000,000"
439,319630,Life is Strange - Episode 1,"5,000,000 .. 10,000,000"
128,41000,Serious Sam HD: The First Encounter,"1,000,000 .. 2,000,000"
859,1213210,Command & Conquer Remastered Collection,"1,000,000 .. 2,000,000"


In [ ]:
df_steamspy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   appid   1000 non-null   int64 
 1   name    1000 non-null   object
 2   owners  1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [ ]:
df_steamspy_0 = df_steamspy.copy()

In [ ]:
df_steamspy["min_number_of_owners"] = (
    df_steamspy["owners"]
    .str.replace(',', '', regex=False) 
    .str.extract(r'(\d+)')            
    .astype(int)                       
)
df_steamspy["max_number_of_owners"] = (
    df_steamspy["owners"]
    .str.replace(',', '', regex=False)
    .str.extract(r'\.\. (\d+)')       
    .astype(int)
)
df_steamspy = df_steamspy.drop('owners', axis = 1)

In [ ]:
df_steamspy.sample(10)

,appid,name,min_number_of_owners,max_number_of_owners
878,1254120,Bless Unleashed,2000000,5000000
373,280790,Creativerse,1000000,2000000
816,1057090,Ori and the Will of the Wisps,2000000,5000000
66,10680,Aliens vs. Predator,1000000,2000000
129,41070,Serious Sam 3: BFE,2000000,5000000
49,6910,Deus Ex: Game of the Year Edition,1000000,2000000
960,1755910,Capcom Arcade 2nd Stadium,1000000,2000000
609,506540,Last Man Standing,2000000,5000000
408,301640,Zombie Army Trilogy,1000000,2000000
911,1371980,No Rest for the Wicked,2000000,5000000
